In [1]:
!pip install --upgrade google-genai google-cloud-modelarmor
# google-genai : Client for interacting with the Gimin models
# google-cloud-modelarmor : The API / Client I am using for the sanitization/relation/security

In [2]:
# --- FORCE AUTHENTICATION ---
!gcloud auth application-default login
# Service Account configuration was leading to conflict so for effeciency bypassing SA and authorizing the notebook to behave based on the personal credentials of the account


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=1mbm8QRe2zyh31oPEabStpKMjooCL7&prompt=consent&token_

In [3]:
# ==============================
# SEGMENT 1: CONFIGURATION
#===============================
import sys
import os

# 1. Project Variables (Avoids Hardcoding)
try:
    PROJECT_ID = "qwiklabs-gcp-03-5dc51bd29ec6"
    REGION = "us-central1"
except:
    PROJECT_ID = "your-project-id"

# 2. Declare/Set Desired Version of Gemini
TARGET_MODEL = "gemini-2.5-pro"
# Per lifecycle using the latest version
# Review: https://docs.cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions

# 3. Define Security Config
TEMPLATE_ID = "StarKiller_MA"
# ^ The specefic security policy I defined to be built into the console
MA_LOCATION = "us-central1"


# --- VALIDATION ---
print(f"Configuration Set:")
print(f"   - Project ID: {PROJECT_ID}")
print(f"   - Region: {REGION}")
print(f"   - Target Model: {TARGET_MODEL}")

Configuration Set:
   - Project ID: qwiklabs-gcp-03-5dc51bd29ec6
   - Region: us-central1
   - Target Model: gemini-2.5-pro


In [4]:
# --- SEGMENT 2: IMPORTS & INIT ---
from google.cloud import aiplatform
from google.cloud import modelarmor_v1
# modelarmor_v1 - Specefic library for reaching the MA security service
import vertexai
# Main libraary for Gemini configs
from vertexai.generative_models import GenerativeModel, SafetySetting, HarmCategory, HarmBlockThreshold
# Parameters MA is going to be watching for by default
import json
# Using this to help parse data, system will utilize this to review the confidence scores and ratings

try:
    # 1. Initialize Vertex AI SDK
    vertexai.init(project=PROJECT_ID, location=REGION)

    # 2. Initialize Model Armor Client
    ma_client = modelarmor_v1.ModelArmorClient()

    # --- VALIDATION ---
    print(" Libraries Imported Successfully.")
    print(f" Vertex AI Initialized for: {PROJECT_ID} @ {REGION}")
    print(" Model Armor Client: Connected")
    #Validation Check

except Exception as e:
    print(f" Initialization Failed: {e}")
    # Fail System Report

 Libraries Imported Successfully.
 Vertex AI Initialized for: qwiklabs-gcp-03-5dc51bd29ec6 @ us-central1
 Model Armor Client: Connected


In [5]:
# --- SEGMENT 3: VERIFY SECURITY INFRASTRUCTURE (DOCUMENTATION ALIGNED) ---
from google.cloud import modelarmor_v1
from google.api_core.client_options import ClientOptions
# Configures connection to operate the the model armor instead of using the default global IP
# Doing this because StarKiller_MA lives only on us-central1

def verify_template_strict(project, location, template_id):
    # Reference: https://atamel.dev/posts/2025/08-11_secure_llm_model_armor/
    # Requirement: Explicitly define the Regional API Endpoint
    api_endpoint = f"modelarmor.{location}.rep.googleapis.com"

    print(f" Connecting to Regional Endpoint: {api_endpoint}")

    try:
        # 1. Initialize Client with specific Regional Options (Per Doc Source 2.3)
        client = modelarmor_v1.ModelArmorClient(
            client_options=ClientOptions(api_endpoint=api_endpoint)
        )

        # 2. Construct Full Resource Name
        template_full_path = f"projects/{project}/locations/{location}/templates/{template_id}"
        print(f" Verifying: {template_full_path}")
        # Converts the variable into a MR / Machine Readable Format

        # 3. Fetch Template
        request = modelarmor_v1.GetTemplateRequest(name=template_full_path)
        existing = client.get_template(request=request)
        #Circuit breaker method -  pinging the MA and seeing if it can run with it, if not prevention of running in an insecure state

        return existing.name, "Found & Verified"

    except Exception as e:
        print(f" Error details: {str(e)}")
        return None, "Connection Failed"

# --- EXECUTE ---
PROJECT_ID = "qwiklabs-gcp-03-5dc51bd29ec6"
MA_LOCATION = "us-central1"
TEMPLATE_ID = "StarKiller_MA"

SECURITY_POLICY_PATH, status = verify_template_strict(PROJECT_ID, MA_LOCATION, TEMPLATE_ID)
# Verifies function and stores the result

if SECURITY_POLICY_PATH:
    print(f" Security Policy Ready: {status}")
    print(f"   Target: {SECURITY_POLICY_PATH}")
else:
    print(" CRITICAL FAIL: Check Project ID and Location.")

 Connecting to Regional Endpoint: modelarmor.us-central1.rep.googleapis.com
 Verifying: projects/qwiklabs-gcp-03-5dc51bd29ec6/locations/us-central1/templates/StarKiller_MA
 Security Policy Ready: Found & Verified
   Target: projects/qwiklabs-gcp-03-5dc51bd29ec6/locations/us-central1/templates/StarKiller_MA


In [6]:
# --- SEGMENT 4: BOT LOGIC CLASS (WITH STRATEGIC BREAKDOWN) ---
from google.api_core.client_options import ClientOptions
import json
import re

class GuidedDefenseBot:
    def __init__(self, project_id, location, template_name, model_id):
        self.template_name = template_name
        self.location = location

        # [SECURITY LAYER] Initialize Model Armor
        # We are forcing the Regional Endpoint here because the global gateway failed us earlier.
        # This links strict layout to the specific StarKiller_MA policy in us-central1.
        api_endpoint = f"modelarmor.{location}.rep.googleapis.com"
        self.ma_client = modelarmor_v1.ModelArmorClient(
            client_options=ClientOptions(api_endpoint=api_endpoint)
        )

        # --- THE AUDITOR (GUARDRAIL MODEL) ---
        # This is where we imprison the AI.
        # By hard-coding these rules into the system instruction, we strip away its ability
        # to be creative or helpful with non-IT topics. It becomes a strict binary judge.
        auditor_rules = """
        You are a specialized Technical Security Auditor.
        Your ONLY function is to validate that user inputs are strictly technically relevant to:
        - Software Engineering
        - IT Infrastructure & Networking
        - Cloud Computing
        - Cyber Security

        CRITICAL RULES:
        1. LITERAL INTERPRETATION: Do NOT interpret metaphors, movies, or pop culture as technical requirements.
        2. NO CREATIVITY: Do NOT "bridge the gap". If Topic is "Cables" and Details are "Star Wars", the relation is FALSE.
        3. FAIL SAFE: If the input is ambiguous or non-technical, reject it.
        """
        self.guardrail_model = GenerativeModel(
            model_id,
            system_instruction=auditor_rules
        )

        # --- THE EXPERT (WORKER MODEL) ---
        # This is the Worker who actually does the job.
        # We separate this from the Auditor so the logic doesn't bleed over.
        # We explicitly forbid PII and Malware generation at the system level.
        expert_rules = """
        You are a Senior Staff Software Engineer and Cloud Architect.
        Your goal is to provide production-grade, secure, and idiomatic technical solutions.

        OUTPUT FORMAT:
        1. Brief Executive Summary
        2. The Technical Solution (Code/Config)
        3. Security Considerations

        SAFETY PROTOCOLS:
        - NEVER generate PII (Personally Identifiable Information).
        - NEVER generate malware, exploit payloads, or reverse shells.
        - If a request is technically dangerous, refuse it.
        """
        self.worker_model = GenerativeModel(
            model_id,
            system_instruction=expert_rules,
            safety_settings=[
                SafetySetting(category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT, threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE)
            ]
        )

    def sanitize_input(self, text):
        """Layer 1: The Armor Check"""
        # Before we even analyze meaning, we check for 'Attacks'.
        # This hits the StarKiller_MA template to detect Jailbreaks or Injection attempts.
        req = modelarmor_v1.SanitizeUserPromptRequest(
            name=self.template_name,
            user_prompt_data=modelarmor_v1.DataItem(text=text)
        )
        resp = self.ma_client.sanitize_user_prompt(request=req)
        match = resp.sanitization_result.filter_match_state == modelarmor_v1.FilterMatchState.MATCH_FOUND
        return (not match), "Blocked by Security Filter (Model Armor)" if match else "Safe"

    def extract_json(self, text):
        # Helper to parse the Auditor's judgment.
        # Includes a regex fallback because LLMs sometimes add polite conversational filler.
        try:
            match = re.search(r'\{.*\}', text, re.DOTALL)
            if match: return json.loads(match.group())
            return json.loads(text)
        except: return None

    def check_topic_validity(self, topic):
        """Layer 1.5: The Semantic Gatekeeper"""
        # Instead of a brittle list of allowed words, we ask the Auditor:
        # "Is this semantically related to our domain?"
        # This stops the 'Pineapples' and 'Cooking' queries immediately.
        prompt = f"""
        Task: Boolean Classification.
        Input: "{topic}"
        Is this technically relevant to the Domain Definition?
        Answer JSON only: {{"is_related": <boolean>}}
        """
        response = self.guardrail_model.generate_content(prompt)
        data = self.extract_json(response.text)
        if data: return data.get("is_related", False)
        return False

    def analyze_alignment(self, topic, details):
        """Layer 2: The Logic Check (Anti-Hallucination)"""
        # This is the "Star Wars" defense.
        # We force the model to compare the Topic vs. Details for technical consistency.
        # If the user tries to sneak in nonsense specifications, this layer kills it.
        prompt = f"""
        Task: Alignment Audit.
        Claimed Topic: "{topic}"
        Provided Details: "{details}"

        Output JSON only: {{"confidence_score": <int 0-100>, "is_safe": <boolean>, "reason": "Short explanation"}}
        """
        response = self.guardrail_model.generate_content(prompt)
        data = self.extract_json(response.text)
        return data if data else {"confidence_score": 0, "is_safe": False, "reason": "Analysis Error"}

    def run_session(self):
        print("\n---  SECURE GUIDED SESSION  ---")

        # --- STEP 1: TOPIC INGESTION ---
        topic = input("AI: What is the specific Coding or IT Topic? > ")

        # Check A: Is it an Attack? (Model Armor)
        safe, msg = self.sanitize_input(topic)
        if not safe: return print(f" BLOCKED: {msg}")

        # Check B: Is it Relevant? (Semantic Gatekeeper)
        # If fails here, we print the Imprisonment message and exit.
        if not self.check_topic_validity(topic):
            print(f"\nAI: I am imprisoned to IT topics only. I cannot assist with [{topic}].")
            return

        print(f"   -> Topic Accepted.")

        # --- STEP 2: DETAILS INGESTION ---
        details = input("AI: Please provide the specifications > ")

        # Check A: Is the detail an Attack? (Model Armor)
        safe, msg = self.sanitize_input(details)
        if not safe: return print(f" BLOCKED: {msg}")

        # --- STEP 3: DEEP ANALYSIS ---
        print("   -> Analyzing Request Alignment...")
        # Check B: Do the details match the topic? (Logic Check)
        analysis = self.analyze_alignment(topic, details)
        print(f"\n Audit Result: Confidence Score {analysis['confidence_score']}%")
        print(f"   Reason: {analysis['reason']}")

        # Threshold Enforcer: If score < 70, we reject.
        if not analysis['is_safe'] or analysis['confidence_score'] < 70:
            print(f"\n Request Rejected: {analysis['reason']}")
            return

        # --- STEP 4: EXECUTION ---
        # Only now do we let the worker model generate the solution.
        print("\n Validated. Generating Response...")
        res = self.worker_model.generate_content(f"Topic: {topic}\nDetails: {details}\nProvide technical solution.")
        print(f"\n{res.text}")

print(" Logic Core Updated (System Instructions & Defense-in-Depth).")

 Logic Core Updated (System Instructions & Defense-in-Depth).


In [ ]:
# --- SEGMENT 5: EXECUTION PROTOCOL ---
import time

if __name__ == "__main__":
    # SAFETY INTERLOCK
    # verify that the Security Policy (StarKiller_MA) was successfully loaded in Segment 3.
    # The system refuses to launch the AI without the containment field active.
    if 'SECURITY_POLICY_PATH' in globals() and SECURITY_POLICY_PATH:

        print(f" SYSTEM INITIALIZATION: Spinning up {TARGET_MODEL}...")
        print(f"  DEFENSE LAYER ACTIVE: Linked to policy '{SECURITY_POLICY_PATH.split('/')[-1]}'")

        try:
            # INSTANTIATION
            # We bind the GuidedDefenseBot to the specific Project and Regional Security Policy.
            # This applies the Imprisonment rules defined in the Class Logic.
            bot = GuidedDefenseBot(PROJECT_ID, MA_LOCATION, SECURITY_POLICY_PATH, TARGET_MODEL)

            # INTERROGATION LOOP
            # This loop keeps the session alive, allowing for rapid-fire testing of
            # multiple attack vectors without reloading the entire kernel.
            while True:
                bot.run_session()

                # SESSION MANAGEMENT
                print("\n" + "="*50)
                if input("Re-engage Neural Net? (y/n): ").strip().lower() != 'y':
                    print(" Protocol Terminated. Security integrity maintained.")
                    break
                print("="*50 + "\n")

        except Exception as e:
            # RUNTIME FAILURE
            print(f"\n CRITICAL ERROR: The containment field failed during execution. {e}")
            print("Tip: Ensure Segment 4 (The Logic Core) was loaded without errors.")

    else:
        # [ABORT PROTOCOL]
        print("CRITICAL FAIL: Security Policy not detected.")
        print("Action: The system refuses to operate in an unsecured state. Return to Segment 3.")

 SYSTEM INITIALIZATION: Spinning up gemini-2.5-pro...
  DEFENSE LAYER ACTIVE: Linked to policy 'StarKiller_MA'


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()



---  SECURE GUIDED SESSION  ---
AI: What is the specific Coding or IT Topic? > Ethernet Ports
   -> Topic Accepted.
AI: Please provide the specifications > I need help troubleshooting ethernet ports on my system
   -> Analyzing Request Alignment...

 Audit Result: Confidence Score 100%
   Reason: The topic 'Ethernet Ports' and the task 'troubleshooting' are core, unambiguous concepts within IT Infrastructure & Networking.

 Validated. Generating Response...

Of course. Here is a production-grade, systematic approach to troubleshooting Ethernet port issues on a local system.

### 1. Brief Executive Summary

This guide provides a systematic, multi-layered methodology for diagnosing and resolving Ethernet connectivity problems. The process begins with the most common and simplest issues at the Physical Layer (cables, lights) and progressively moves up the stack to the Operating System (drivers, interface status), Network Configuration (IP addressing, DNS), and finally, external connectiv